In [18]:
trade_map = { "I": "orig_id"
  ,
  "x":"exchange",

  "p":"price",
  
  
  "i":"id",
   
  "e":"correction",

  "r":"trf_id",

  "t":"sip_timestamp",

  "y":"participant_timestamp",

  "f":"trf_timestamp",

  "q":"sequence_number",

  "c":"conditions",

  "s":"size",

  "z":"tape"
 }

In [19]:
trade_conditions = {'0': 'Regular', '1': 'Acquisition', '2': 'AveragePrice', '3': 'AutomaticExecution', '4': 'Bunched', '5': 'BunchSold', '6': 'CAPElection', '7': 'CashTrade', '8': 'Closing', '9': 'Cross', '10': 'DerivativelyPriced', '11': 'Distribution', '12': 'FormT(ExtendedHours)', '13': 'FormTOutOfSequence', '14': 'InterMarketSweep', '15': 'MarketCenterOfficialClose', '16': 'MarketCenterOfficialOpen', '17': 'MarketCenterOpening', '18': 'MarketCenterReOpenning', '19': 'MarketCenterClosing', '20': 'NextDay', '21': 'PriceVariation', '22': 'PriorReferencePrice', '23': 'Rule155Amex', '24': 'Rule127Nyse', '25': 'Opening', '26': 'Opened', '27': 'RegularStoppedStock', '28': 'ReOpening', '29': 'Seller', '30': 'SoldLast', '31': 'SoldLastStoppedStock', '32': 'SoldOutOfSequence', '33': 'SoldOutOfSequenceStoppedStock', '34': 'Split', '35': 'StockOption', '36': 'YellowFlag', '37': 'OddLot', '38': 'CorrectedConsolidatedClosePrice', '39': 'Unknown', '40': 'Held', '41': 'TradeThruExempt', '42': 'NonEligible', '43': 'NonEligible-extended', '44': 'Cancelled', '45': 'Recovery', '46': 'Correction', '47': 'AsOf', '48': 'AsOfCorrection', '49': 'AsOfCancel', '50': 'OOB', '51': 'Summary', '52': 'Contingent', '53': 'Contingent(Qualified)', '54': 'Errored'}
quote_conditions = {'0': 'Regular', '1': 'RegularTwoSidedOpen', '2': 'RegularOneSidedOpen', '3': 'SlowAsk', '4': 'SlowBid', '5': 'SlowBidAsk', '6': 'SlowDueLRPBid', '7': 'SlowDueLRPAsk', '8': 'SlowDueNYSELRP', '9': 'SlowDueSetSlowListBidAsk', '10': 'ManualAskAutomaticBid', '11': 'ManualBidAutomaticAsk', '12': 'ManualBidAndAsk', '13': 'Opening', '14': 'Closing', '15': 'Closed', '16': 'Resume', '17': 'FastTrading', '18': 'TradingRangeIndication', '19': 'MarketMakerQuotesClosed', '20': 'NonFirm', '21': 'NewsDissemination', '22': 'OrderInflux', '23': 'OrderImbalance', '24': 'DueToRelatedSecurityNewsDissemination', '25': 'DueToRelatedSecurityNewsPending', '26': 'AdditionalInformation', '27': 'NewsPending', '28': 'AdditionalInformationDueToRelatedSecurity', '29': 'DueToRelatedSecurity', '30': 'InViewOfCommon', '31': 'EquipmentChangeover', '32': 'NoOpenNoResume', '33': 'SubPennyTrading', '34': 'AutomatedBidNoOfferNoBid', '35': 'LuldPriceBand', '36': 'MarketWideCircuitBreakerLevel1', '37': 'MarketWideCircuitBreakerLevel2', '38': 'MarketWideCircuitBreakerLevel3', '39': 'RepublishedLuldPriceBand', '40': 'OnDemandAuction', '41': 'CashOnlySettlement', '42': 'NextDaySettlement', '43': 'LULDTradingPause', '71': 'SlowDueLRPBidAsk'}

In [20]:
import datetime
import pandas as pd
import pytz
import time
eastern = 'US/Eastern'

from polygon import RESTClient

In [21]:
key = "qRFktbNL6A_WzaFhqR80IllApsmjPY6a"

def ts_to_datetime(ts) -> str:
    return datetime.datetime.fromtimestamp(ts / 1000.0).strftime('%Y-%m-%d %H:%M')

def get_list_of_Exchanges():

    with RESTClient(key) as client:
        respex = client.stocks_equities_exchanges()

        stock_exchanges = respex.exchange
        exchange_list_dict = {}

        for i in stock_exchanges:
            the_dict = {}
            the_dict[i.i_d_of_the_exchange] = i.name
            #exchange_list_dict.append(the_dict)
            exchange_list_dict[i.i_d_of_the_exchange] = i.name
        return exchange_list_dict

def get_list_of_trade_conditions():

    with RESTClient(key) as client:
        respex = client.stocks_equities_condition_mappings()

        stock_exchanges = respex.exchange
        exchange_list_dict = {}

        for i in stock_exchanges:
            the_dict = {}
            the_dict[i.i_d_of_the_exchange] = i.name
            #exchange_list_dict.append(the_dict)
            exchange_list_dict[i.i_d_of_the_exchange] = i.name
        return exchange_list_dict
    
def update_condition_labels(keyFunction, values):
    #if type(list(values)) == list:
    k =  list(keyFunction[str(v)] for v in list(values))
    return ", ".join(k)
    #return values

In [22]:
def get_trades_all(ticker, date, size):
    total_data = []
    lasttimestamp = 0
    
    while True:
        with RESTClient(key) as client:
            
            if lasttimestamp == 0:
                resp = client.historic_trades_v2(ticker=str(ticker).upper(), date=date, limit=size, reverse=False)
            else:
                resp = client.historic_trades_v2(ticker=str(ticker).upper(), date=date, timestamp = lasttimestamp, limit=size, reverse=False)

            result = resp.__dict__
            
            if "results" in result:
                result = result['results']
            else:
                break
                
            lasttimestamp = result[-1]['t']
            total_data.extend(result)
            
            if len(result) == 50000:
                continue
            else:
                break
        
        time.sleep(1)
        
    return total_data

In [23]:
trades = get_trades_all("AAPL", "2021-02-17", 500000)

In [24]:

trades_df = pd.DataFrame(trades).rename(columns=trade_map)
trades_df['sip_timestamp'] =pd.to_datetime(trades_df['sip_timestamp']).dt.tz_localize('utc').dt.tz_convert(eastern)
trades_df['participant_timestamp'] =pd.to_datetime(trades_df['participant_timestamp']).dt.tz_localize('utc').dt.tz_convert(eastern)

#### CHECK IF FULL PREMARKET DATA EXITS

In [25]:
trades_df['hours'] = trades_df['sip_timestamp'].apply(lambda x: x.hour)
trades_df['minutes'] = trades_df['sip_timestamp'].apply(lambda x: x.minute)
trades_df['seconds'] = trades_df['sip_timestamp'].apply(lambda x: x.second)

In [26]:
new_new_df = trades_df[(trades_df['hours'] == 9) & (trades_df['minutes'] > 28) & (trades_df['minutes'] < 31)]

In [29]:
new_new_df.head(1000)

,sip_timestamp,participant_timestamp,sequence_number,id,exchange,size,conditions,price,tape,trf_timestamp,trf_id,correction,hours,minutes,seconds
40749,2021-02-17 09:29:00.276340515-05:00,2021-02-17 09:29:00.275972352-05:00,322642,11148,11,67,"[14, 12, 37, 41]",131.2100,3,NaN,NaN,NaN,9,29,0
40750,2021-02-17 09:29:00.276340667-05:00,2021-02-17 09:29:00.275972352-05:00,322643,11149,11,33,"[14, 12, 37, 41]",131.2100,3,NaN,NaN,NaN,9,29,0
40751,2021-02-17 09:29:00.875203831-05:00,2021-02-17 09:29:00.874839040-05:00,322660,11150,11,5,"[14, 12, 37, 41]",131.2100,3,NaN,NaN,NaN,9,29,0
40752,2021-02-17 09:29:01.098869469-05:00,2021-02-17 09:29:01.098504704-05:00,322672,11151,11,62,"[14, 12, 37, 41]",131.2100,3,NaN,NaN,NaN,9,29,1
40753,2021-02-17 09:29:01.098871509-05:00,2021-02-17 09:29:01.098504704-05:00,322673,11152,11,38,"[14, 12, 37, 41]",131.2100,3,NaN,NaN,NaN,9,29,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41744,2021-02-17 09:30:01.640468384-05:00,2021-02-17 09:30:01.638000-05:00,332877,6572,4,2,[37],131.4000,3,1.613572e+18,12.0,NaN,9,30,1
41745,2021-02-17 09:30:01.642269659-05:00,2021-02-17 09:30:01.640000-05:00,332896,6573,4,1,[37],131.4000,3,1.613572e+18,12.0,NaN,9,30,1
41746,2021-02-17 09:30:01.644491330-05:00,2021-02-17 09:30:01.644474815-05:00,332911,10932,12,1,"[14, 12, 37, 41]",131.3600,3,NaN,NaN,NaN,9,30,1
41747,2021-02-17 09:30:01.649211231-05:00,2021-02-17 09:30:01.647000-05:00,332946,6574,4,3021,NaN,131.4389,3,1.613572e+18,12.0,NaN,9,30,1
